In [1]:
import pandas as pd
import io
import json

In [2]:
df = pd.read_csv("../data/raw/train_data_ner.csv")
print(df.head())

   sentence_id  token      label
0            1    Tôi          O
1            1     bị          O
2            1    đau  B-SYMPTOM
3            1    đầu  I-SYMPTOM
4            1     và          O


In [3]:
def convert_to_spacy_format(data_string):
    # Đọc dữ liệu từ string
    data_io = io.StringIO(data_string)
    df = pd.read_csv(data_io)
    
    # Loại bỏ khoảng trắng thừa từ tên cột
    df.columns = df.columns.str.strip()
    
    # Loại bỏ các dòng NaN (nếu có)
    df = df.dropna(subset=['sentence_id', 'token', 'label'])
    
    spacy_data = []
    
    # Nhóm theo sentence_id
    grouped = df.groupby('sentence_id')
    
    for _, group in grouped:
        tokens = group['token'].tolist()
        labels = group['label'].tolist()
        
        raw_text = ""
        token_offsets = [] # Lưu (start_char, end_char) cho mỗi token
        
        current_char = 0
        for token in tokens:
            token = str(token) # Đảm bảo token là string
            start = current_char
            end = start + len(token)
            token_offsets.append((start, end))
            
            raw_text += token + " "
            current_char = end + 1 # +1 cho dấu cách
            
        raw_text = raw_text.strip() # Xóa dấu cách cuối cùng
        
        entities = []
        
        current_entity_label = None
        current_entity_start_token_idx = None
        
        for i, label in enumerate(labels):
            if label.startswith("B-"):
                # Nếu đang có 1 thực thể, lưu nó lại trước
                if current_entity_label:
                    start_char = token_offsets[current_entity_start_token_idx][0]
                    # Vị trí kết thúc là vị trí kết thúc của token (i-1)
                    end_char = token_offsets[i - 1][1]
                    entities.append((start_char, end_char, current_entity_label))
                
                # Bắt đầu thực thể mới
                current_entity_label = label.split("-")[1]
                current_entity_start_token_idx = i
            
            elif label.startswith("I-"):
                # Nếu token I- không khớp với B- (lỗi gán nhãn)
                if not current_entity_label or label.split("-")[1] != current_entity_label:
                    # Bỏ qua token I- lỗi này và đóng thực thể cũ nếu có
                    if current_entity_label:
                        start_char = token_offsets[current_entity_start_token_idx][0]
                        end_char = token_offsets[i - 1][1]
                        entities.append((start_char, end_char, current_entity_label))
                    current_entity_label = None
                    current_entity_start_token_idx = None
            
            elif label == "O":
                # Nếu gặp 'O', đóng thực thể hiện tại (nếu có)
                if current_entity_label:
                    start_char = token_offsets[current_entity_start_token_idx][0]
                    end_char = token_offsets[i - 1][1]
                    entities.append((start_char, end_char, current_entity_label))
                current_entity_label = None
                current_entity_start_token_idx = None
        
        # Xử lý thực thể cuối cùng trong câu
        if current_entity_label:
            start_char = token_offsets[current_entity_start_token_idx][0]
            end_char = token_offsets[len(tokens) - 1][1] # Lấy vị trí cuối của token cuối cùng
            entities.append((start_char, end_char, current_entity_label))
            
        spacy_data.append((raw_text, {"entities": entities}))
        
    return spacy_data

In [4]:

# Chạy chuyển đổi
TRAIN_DATA = convert_to_spacy_format(df.to_csv(index=False))

# In kết quả
print("DANH SÁCH DỮ LIỆU HUẤN LUYỆN (ĐỊNH DẠNG SPACY):\n")
for item in TRAIN_DATA:
    print(item)

DANH SÁCH DỮ LIỆU HUẤN LUYỆN (ĐỊNH DẠNG SPACY):

('Tôi bị đau đầu và sốt nhẹ', {'entities': [(7, 14, 'SYMPTOM'), (18, 25, 'SYMPTOM')]})
('Mấy hôm nay tôi ho khan và đau họng', {'entities': [(16, 23, 'SYMPTOM'), (27, 35, 'SYMPTOM')]})
('Tôi bị sổ mũi và nghẹt mũi mấy ngày nay', {'entities': [(7, 13, 'SYMPTOM'), (17, 26, 'SYMPTOM')]})
('Cơ thể tôi rất mệt mỏi và uể oải', {'entities': [(15, 22, 'SYMPTOM'), (26, 32, 'SYMPTOM')]})
('Tôi có triệu chứng ớn lạnh và đau nhức toàn thân', {'entities': [(19, 26, 'SYMPTOM'), (30, 48, 'SYMPTOM')]})
('Sốt cao và ho có phải là triệu chứng của bệnh cảm cúm không', {'entities': [(0, 7, 'SYMPTOM'), (11, 13, 'SYMPTOM'), (46, 53, 'DISEASE')]})
('Cháu nhà tôi bị chảy nước mũi liên tục', {'entities': [(16, 29, 'SYMPTOM')]})
('Ngoài đau họng tôi còn bị mất vị giác', {'entities': [(6, 14, 'SYMPTOM'), (26, 37, 'SYMPTOM')]})
('Tôi cảm thấy mệt mỏi và buồn nôn', {'entities': [(13, 20, 'SYMPTOM'), (24, 32, 'SYMPTOM')]})
('Con tôi bị sốt cao và rét run', {'entities

In [5]:
# --- PHẦN MỚI: LƯU VÀO TỆP .JSONL ---
output_file = "../data/processed/train_data.spacy.jsonl"
count = 0

try:
    with open(output_file, 'w', encoding='utf-8') as f:
        for text, annotations in TRAIN_DATA:
            # Tạo đối tượng JSON cho mỗi dòng
            # Định dạng này hơi khác một chút, chỉ lưu text và các thực thể
            # để dễ dàng đọc bằng các thư viện khác.
            # Định dạng (text, {"entities": [...]}) là cho code Python, 
            # còn định dạng JSONL thường là {"text": ..., "entities": ...}
            line_data = {"text": text, "entities": annotations['entities']}
            
            # Ghi dòng JSON vào tệp
            f.write(json.dumps(line_data, ensure_ascii=False) + '\n')
            count += 1

    print(f"Đã lưu thành công {count} dòng vào tệp '{output_file}'")

except Exception as e:
    print(f"Đã xảy ra lỗi khi lưu tệp: {e}")

Đã lưu thành công 20 dòng vào tệp '../data/processed/train_data.spacy.jsonl'
